In [10]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os

In [11]:
def read_data(input_file):
    rf = open(input_file, 'r')
    lines = []; words = []; labels = []
    for line in rf:
        word = line.strip().split(' ')[0]
        label = line.strip().split(' ')[-1]
        if len(line.strip()) == 0 and words[-1] == '.':
            l = ' '.join([label for label in labels if len(label) > 0])
            w = ' '.join([word for word in words if len(word) > 0])
            lines.append((l, w))
            words = []
            labels = []
        words.append(word)
        labels.append(label)
    labels_list=[]
    for l in lines:
        labels_list.append(l[0])
    return labels_list, lines
   
og_labels, og_lines =  read_data("./baselines/bert_baseline/original_conll03/original_test_predictions.txt")
prop_labels, prop_lines =  read_data("./baselines/bert_baseline/proposed_conll03/proposed_test_predictions.txt")


In [12]:
"""
finding incongruent sequences like B-PER I-LOC
currently only works for entities with two words
"""
def incongruent_label(labels):
    incong=[]
    for l in labels:
        ll=l.split(' ')
        for w in range(len(ll)):
            if ll[w]=='O':
                pass
            else:
                if ll[w].split('-')[0]=='B':
                    if ll[w+1].split('-')[0]=='I':
                        if  ll[w].split('-')[1]!=ll[w+1].split('-')[1]:
                            incong.append((ll[w],ll[w+1]))
    return incong

og_incongruent=incongruent_label(og_labels)
prop_incongruent=incongruent_label(prop_labels)
print(len(og_incongruent),og_incongruent)

print(len(prop_incongruent),prop_incongruent)

52 [('B-ORG', 'I-PER'), ('B-ORG', 'I-PER'), ('B-PER', 'I-ORG'), ('B-LOC', 'I-MISC'), ('B-ORG', 'I-MISC'), ('B-ORG', 'I-MISC'), ('B-LOC', 'I-MISC'), ('B-LOC', 'I-MISC'), ('B-ORG', 'I-MISC'), ('B-LOC', 'I-ORG'), ('B-ORG', 'I-MISC'), ('B-PER', 'I-ORG'), ('B-ORG', 'I-PER'), ('B-LOC', 'I-MISC'), ('B-MISC', 'I-ORG'), ('B-ORG', 'I-MISC'), ('B-ORG', 'I-LOC'), ('B-LOC', 'I-ORG'), ('B-MISC', 'I-LOC'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-MISC'), ('B-ORG', 'I-LOC'), ('B-PER', 'I-LOC'), ('B-PER', 'I-LOC'), ('B-MISC', 'I-ORG'), ('B-LOC', 'I-PER'), ('B-LOC', 'I-ORG'), ('B-MISC', 'I-LOC'), ('B-MISC', 'I-ORG'), ('B-LOC', 'I-MISC'), ('B-MISC', 'I-ORG'), ('B-MISC', 'I-ORG'), ('B-MISC', 'I-ORG'), ('B-PER', 'I-ORG'), ('B-PER', 'I-LOC'), ('B-MISC', 'I-ORG'), ('B-LOC', 'I-MISC'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-ORG'), ('B-LOC', 'I-ORG'), ('B-MISC', 'I-LOC'), ('B-MISC', 'I-LOC'), ('B-ORG', 'I-MISC'), ('B-LOC', 'I-MISC'), ('B-ORG', 'I-MISC'), ('B-LOC', 

In [13]:
import csv
from itertools import groupby
def read_conll(filename, columns, delimiter='\t'):
    def is_empty_line(line_pack):
        return all(field.strip() == '' for field in line_pack)
    data = []
    with open(filename) as fp:
        reader = csv.reader(fp, delimiter=delimiter, quoting=csv.QUOTE_NONE)
        groups = groupby(reader, is_empty_line)
        for is_empty, pack in groups:
            if is_empty is False:
                data.append([list(field) for field in zip(*pack)])
    data = list(zip(*data))
    dataset = {colname: list(data[columns[colname]]) for colname in columns}
    return dataset

In [14]:
og_pred_filename = "./baselines/bert_baseline/original_conll03/original_test_predictions.txt"
prop_pred_filename = "./baselines/bert_baseline/proposed_conll03/proposed_test_predictions.txt"
columns = {"Tokens": 0, "Preds": 1}
delimiter = ' '
og_pred_data = read_conll(og_pred_filename, columns, delimiter)
prop_pred_data = read_conll(prop_pred_filename, columns, delimiter)

In [15]:
og_pred_df=pd.DataFrame(og_pred_data)
prop_pred_df=pd.DataFrame(prop_pred_data)

In [16]:
og_pred_df

,Tokens,Preds
0,"[SOCCER, -, JAPAN, GET, LUCKY, WIN, ,, CHINA, ...","[O, O, B-LOC, O, O, O, O, B-LOC, O, O, O, O]"
1,"[Nadim, Ladki]","[B-PER, I-PER]"
2,"[AL-AIN, ,, United, Arab, Emirates, 1996-12-06]","[B-LOC, O, B-LOC, I-LOC, I-LOC, O]"
3,"[Japan, began, the, defence, of, their, Asian,...","[B-LOC, O, O, O, O, O, B-MISC, I-MISC, O, O, O..."
4,"[But, China, saw, their, luck, desert, them, i...","[O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O,..."
5,"[China, controlled, most, of, the, match, and,...","[B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
6,"[Oleg, Shatskiku, made, sure, of, the, win, in...","[B-PER, I-PER, O, O, O, O, O, O, O, O, O, O, O..."
7,"[The, former, Soviet, republic, was, playing, ...","[O, O, B-MISC, O, O, O, O, O, B-MISC, I-MISC, ..."
8,"[Despite, winning, the, Asian, Games, title, t...","[O, O, O, B-MISC, I-MISC, O, O, O, O, O, B-LOC..."
9,"[Two, goals, from, defensive, errors, in, the,...","[O, O, O, O, O, O, O, O, O, O, O, B-LOC, O, O,..."
